In [41]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor

from sklearn.tree import export_graphviz
import pydot

In [2]:
df = pd.read_excel('/content/temps.xlsx')

In [3]:
df.head()

,year,month,day,week,temp_2,temp_1,average,actual
0,2016,1,1,Fri,45,45,45.6,45
1,2016,1,2,Sat,44,45,45.7,44
2,2016,1,3,Sun,45,44,45.8,41
3,2016,1,4,Mon,44,41,45.9,40
4,2016,1,5,Tues,41,40,46.0,44


In [4]:
df.describe()

,year,month,day,temp_2,temp_1,average,actual
count,348.0,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000
mean,2016.0,6.477011,15.514368,62.652299,62.701149,59.760632,62.543103
std,0.0,3.498380,8.772982,12.165398,12.120542,10.527306,11.794146
min,2016.0,1.000000,1.000000,35.000000,35.000000,45.100000,35.000000
25%,2016.0,3.000000,8.000000,54.000000,54.000000,49.975000,54.000000
50%,2016.0,6.000000,15.000000,62.500000,62.500000,58.200000,62.500000
75%,2016.0,10.000000,23.000000,71.000000,71.000000,69.025000,71.000000
max,2016.0,12.000000,31.000000,117.000000,117.000000,77.400000,92.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   year     348 non-null    int64  
 1   month    348 non-null    int64  
 2   day      348 non-null    int64  
 3   week     348 non-null    object 
 4   temp_2   348 non-null    int64  
 5   temp_1   348 non-null    int64  
 6   average  348 non-null    float64
 7   actual   348 non-null    int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 21.9+ KB


In [6]:
df = pd.get_dummies(df)
df.head()

,year,month,day,temp_2,temp_1,average,actual,week_Fri,week_Mon,week_Sat,week_Sun,week_Thurs,week_Tues,week_Wed
0,2016,1,1,45,45,45.6,45,1,0,0,0,0,0,0
1,2016,1,2,44,45,45.7,44,0,0,1,0,0,0,0
2,2016,1,3,45,44,45.8,41,0,0,0,1,0,0,0
3,2016,1,4,44,41,45.9,40,0,1,0,0,0,0,0
4,2016,1,5,41,40,46.0,44,0,0,0,0,0,1,0


In [43]:
#OUTRO MODO DE FAZER X E Y

labels = np.array(df['actual'])
features = df.drop('actual',axis=1)

feature_list = list(features.columns)

features = np.array(features)

In [10]:
x_train,x_test,y_train,y_test = train_test_split(features,labels,test_size=0.25,random_state=42)

In [13]:
# ANALISANDO A MÉDIA PARA COMPARAR DEPOIS COM O MODELO
baseline = x_test[:,feature_list.index('average')]
baseline_error = abs(baseline - y_test)

print('Baseline error: ', round(np.mean(baseline_error),2))

Baseline error:  5.06


In [16]:
rf = RandomForestRegressor(n_estimators=1000,random_state=42)
rf.fit(x_train,y_train)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [17]:
prediction_rf = rf.predict(x_test)

In [27]:
error_rf = abs(prediction_rf - y_test)


In [30]:
r_sq = rf.score(features,labels)
print('R^2',r_sq)
print('MAE', metrics.mean_absolute_error(y_test,prediction_rf))
print('MSE', metrics.mean_squared_error(y_test,prediction_rf))
#COMO O MAE É MENOR QUE A MÉDIA, O MODELO É MELHOR QUE A MÉDIA

R^2 0.932094797587982
MAE 3.932057471264368
MSE 26.68358100000001


In [31]:
ada = AdaBoostRegressor(n_estimators=100,random_state=42)
ada.fit(x_train,y_train)

AdaBoostRegressor(n_estimators=100, random_state=42)

In [32]:
prediction_ada = ada.predict(x_test)

In [34]:
error_ada = abs(prediction_ada - y_test)

In [33]:
r_sq = ada.score(features,labels)
print('R^2',r_sq)
print('MAE', metrics.mean_absolute_error(y_test,prediction_ada))
print('MSE', metrics.mean_squared_error(y_test,prediction_ada))


R^2 0.8774391274404556
MAE 3.6497778742170652
MSE 23.25109372681773


In [35]:
gb = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb.fit(x_train,y_train)

GradientBoostingRegressor(random_state=42)

In [36]:
prediction_gb = gb.predict(x_test)

In [37]:
error_gb = abs(prediction_gb - y_test)

In [38]:
r_sq = gb.score(features,labels)
print('R^2',r_sq)
print('MAE', metrics.mean_absolute_error(y_test,prediction_gb))
print('MSE', metrics.mean_squared_error(y_test,prediction_gb))

R^2 0.9176135011715985
MAE 4.079400899619871
MSE 28.497263451208553


In [39]:
rf2 = RandomForestRegressor(max_depth=3)
rf2.fit(x_train,y_train)

RandomForestRegressor(max_depth=3)

In [40]:
tree = rf2.estimators_[5]

tree

DecisionTreeRegressor(max_depth=3, max_features=1.0, random_state=644028140)

In [47]:
export_graphviz(tree,out_file='tree.dot',feature_names=feature_list,rounded=True,precision=1)
(graph,)= pydot.graph_from_dot_file('tree.dot')


In [48]:
graph.write_png('tree.png')

In [52]:
#FAZENDO A IMPORTANCIA DAS VARIAVEIS
importances = list(rf2.feature_importances_)

feature_importance = [(feature,round(importance,2)) for feature, importance in zip(feature_list,importances)]

feature_importance = sorted(feature_importance,key=lambda x:x[1],reverse=True)
[print('feature:{:20} Importance {}'.format(*pair)) for pair in feature_importance]

feature:temp_1               Importance 0.78
feature:average              Importance 0.2
feature:month                Importance 0.01
feature:temp_2               Importance 0.01
feature:year                 Importance 0.0
feature:day                  Importance 0.0
feature:week_Fri             Importance 0.0
feature:week_Mon             Importance 0.0
feature:week_Sat             Importance 0.0
feature:week_Sun             Importance 0.0
feature:week_Thurs           Importance 0.0
feature:week_Tues            Importance 0.0
feature:week_Wed             Importance 0.0


[None, None, None, None, None, None, None, None, None, None, None, None, None]